In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cvxpy as cp

In [3]:
class SVM:
    def __init__(self, kernel='linear', C=1e6):
        self.kernel = kernel
        self.C = C

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Variables de support
        alpha = cp.Variable(n_samples)

        # Problème d'optimisation
        if self.kernel == 'linear':
            K = X @ X.T  # Noyau linéaire
        elif self.kernel == 'rbf':
            gamma = 1.0 / (2 * np.var(X)) 
            K = np.exp(-gamma * np.sum((X[:, np.newaxis] - X) ** 2, axis=2))
        
        constraints = [0 <= alpha, alpha <= self.C, y * (K @ alpha) >= 1]
        obj = cp.Maximize(np.sum(alpha) - 0.5 * cp.quad_form(alpha, K))
        prob = cp.Problem(obj, constraints)
        prob.solve()

        # Trouver les vecteurs de support
        support_vectors = np.where((alpha.value > 1e-5) & (alpha.value < self.C - 1e-5))[0]
        self.support_vectors = X[support_vectors]
        self.support_labels = y[support_vectors]
        self.alpha = alpha.value[support_vectors]

    def predict(self, X):
        if self.kernel == 'linear':
            K = X @ self.support_vectors.T
        elif self.kernel == 'rbf':
            gamma = 1.0 / (2 * np.var(X))  # Utiliser le même gamma que lors de l'entraînement
            K = np.exp(-gamma * np.sum((X[:, np.newaxis] - self.support_vectors) ** 2, axis=2))
        
        return np.sign(K @ (self.alpha * self.support_labels))

